In [1]:
!pip install pdfMiner

  Running setup.py bdist_wheel for pdfMiner: started
  Running setup.py bdist_wheel for pdfMiner: finished with status 'done'
  Stored in directory: C:\Users\willm_000\AppData\Local\pip\Cache\wheels\09\f3\78\6c04430be0418c440f389337b25b835f731ae2b6b42a925a49
Successfully built pdfMiner


In [72]:
class Question:
    nbr = -1
    dsc = ""
    qid = ""
    
    def __init__(self, nbr, dsc, qid):
        self.nbr = nbr
        self.dsc = dsc
        self.qid = qid

class Section:
    idx = -1
    title = ""
    dsc = ""
    questions = []
    
    def __init__(self, idx, title, dsc):
        self.idx = idx
        self.title = title
        self.dsc = dsc;
        self.questions = []
        
    def addQuestion(self, question):
        if len(self.questions) > 0:
            lastQuestion = self.questions[len(self.questions) - 1]
            if (lastQuestion.nbr + 1 != question.nbr):
                raise ValueError("Questions not in sequence")
        
        self.questions.append(question)
        
class StudyDoc:
    docNm = ""
    
    def __init__(self, docNm):
        self.docNm = docNm

In [128]:
class SectionBuilder:
    sections = []
    pendingSec = None
    pendingQ = None
    
    def __init__(self):
        self.sections = []
        self.state = '?'
        pendingSec = None
        pendingQ = None
        
    def pushSecNm(self, idx, nm):
        print(idx + " " + nm)
        if self.pendingSec:
            raise ValueError("Have a pending section")
        if self.pendingQ:
            raise ValueError("Have a pending question")
        self.pendingSec = Section(idx, nm, "")
        
    def pushSecDsc(self, dsc):
        print(dsc)
        if self.pendingSec == None:
            raise ValueError("No pending section")
        if self.pendingQ:
            raise ValueError("Have a pending question")
        self.pendingSec.dsc = dsc
        self.sections.append(self.pendingSec)
        self.pendingSec = None
        
    def pushQNm(self, idx, nm):
        print(idx + " " + nm)
        if self.pendingQ:
            raise ValueError("Have a pending question")
        if self.pendingSec:
            raise ValueError("Have a pending section")
        self.pendingQ = Question(idx, nm, "")
        
    def pushQid(self, qid):
        print(qid)
        if self.pendingSec:
            raise ValueError("Have a pending section")
#         if self.pendingQ == None:
#             print(qid)
#             raise ValueError("No pending question")
        if self.pendingQ:
            self.pendingQ.qid = qid

            self.sections[len(self.sections) - 1].questions.append(self.pendingQ)

            self.pendingQ = None
        
    def noOpenObj(self):
        return self.pendingSec == None and self.pendingQ == None
    
    def appendObj(self, txt):
#         if self.pendingSec:
#             self.pendingSec.dsc = self.pendingSec.dsc + txt
#         elif self.pendingQ:
#             self.pendingQ.dsc = self.pendingQ.dsc + txt
        if len(self.sections) > 0:
            sec = self.sections[len(self.sections) - 1]
            if len(sec.questions) > 0:
                q = sec.questions[len(sec.questions) - 1]
                q.dsc = q.dsc + txt
            else:
                sec.dsc = sec.dsc + txt
        
    def pushTxt(self, txt):
#         if self.noOpenObj():
#             self.appendObj(txt)
#         else:
#         print(txt)
#         print(":".join("{:02x}".format(ord(c)) for c in txt))
        m = re.search("Section (\d):(.+)", txt)
        if m:
            self.pushSecNm(m.groups()[0], m.groups()[1])
        else:
            m = re.search("Section (\d).(.+)", txt)
            if m:
                self.pushSecDsc(m.groups()[0])
            else:
                m = re.search("(\d)\. (.+)", txt)
                if m:
                    self.pushQNm(m.groups()[0], m.groups()[1])
                else:
                    m = re.search("(H\d.+)", txt)
                    if m:
                        self.pushQid(m.groups()[0])
                    elif self.noOpenObj():
                        self.appendObj(txt)
   

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

rsrcmgr = PDFResourceManager()
retstr = StringIO()
codec = 'utf-8'
laparams = LAParams()
device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
password = ""
maxpages = 0
caching = True
pagenos=set()

fp = open(r'C:\dev\data\health\ICPSR_21600\DS0022\21600-0022-Codebook_Questionnaire.pdf', 'rb')

text = ""
idx = 0
tIdx = 1
for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
    interpreter.process_page(page)

In [8]:
class Question:
    nbr = -1
    dsc = ""
    qid = ""
    
    def __init__(self, nbr, dsc):
        self.nbr = nbr
        self.dsc = dsc
        self.qid = ""

class Section:
    idx = -1
    title = ""
    dsc = ""
    questions = []
    
    def __init__(self, idx, title):
        self.idx = idx
        self.title = title
        self.dsc = "";
        self.questions = []
        
    def putQuestion(self, nbr, dsc):
#         if len(self.questions) > 0:
#             lastQuestion = self.questions[len(self.questions) - 1]
#             if (lastQuestion.nbr + 1 != question.nbr):
#                 raise ValueError("Questions not in sequence")
        self.questions.append(Question(nbr, dsc))
        
    def putQid(self, qid):
        if len(self.questions) > 0:
            lastQ = self.questions[len(self.questions) - 1]
            if len(lastQ.qid) > 0:
                lastQ.qid = lastQ.qid + "; " + qid
            else:
                lastQ.qid = qid
        
class StudyDoc:
    docNm = ""
    sections = []
    
    def __init__(self):
        self.docNm = ""
        self.section = []
        
    def putSection(self, idx, title):
        self.sections.append(Section(idx, title))
        
    def putQuestion(self, nbr, dsc):
        if len(self.sections) > 0:
            lastSec = self.sections[len(self.sections) - 1]
            lastSec.putQuestion(nbr, dsc)
            
    def putQid(self, qid):
        if len(self.sections) > 0:
            lastSec = self.sections[len(self.sections) - 1]
            lastSec.putQid(qid)

In [11]:
import re

def makeMatches(txt):
    idx = 0
    
    doc = StudyDoc()
    
    s = re.search("Section (\d+):(.+)", txt)
    q = re.search("(\d+)\. (.+)", txt)
    d = re.search("(H\d.+)", txt)
    
    while(s or q or d):
        slct = getEarliestMatch(s, q, d)
        
        if slct[0] == 's':
            print("SECTION", s.groups()[0], s.groups()[1])
#         elif slct[0] == 'q':
#             print(q.groups()[0], q.groups()[1])
#         else:
#             print(d.groups()[0])
        
        idx = slct[1].end()
        txt = txt[idx:]
        
        s = re.search("Section (\d+):(.+)", txt)
        q = re.search("(\d+)\. (.+)", txt)
        d = re.search("(H\d.+)", txt)
        
def getEarliestMatch(s, q, d):
    sIdx = 9999999    
    qIdx = 9999999
    dIdx = 9999999

    if s:
        sIdx = s.start()
    if q:
        qIdx = q.start()
    if d:
        dIdx = d.start()
        
    if dIdx <= sIdx and dIdx <= qIdx:
        return ("d", d)
    elif qIdx <= sIdx:
        return ("q", q)
    else:
        return ("s", s)
    
#         if m:
#             self.pushSecNm(m.groups()[0], m.groups()[1])
#         else:
#             m = re.search("Section (\d).(.+)", txt)
#             if m:
#                 self.pushSecDsc(m.groups()[0])
#             else:
#                 m = re.search("(\d)\. (.+)", txt)
#                 if m:
#                     self.pushQNm(m.groups()[0], m.groups()[1])
#                 else:
#                     m = re.search("(H\d.+)", txt)

In [12]:
makeMatches(retstr.getvalue())

('SECTION', '1', ' Overview and Demographics ')
('SECTION', '2', ' Parental Support and Relationships ')
('SECTION', '3', ' Relationships with Siblings ')
('SECTION', '4', ' General Health and Diet ')
('SECTION', '5', ' Access to Health Services, Health Insurance ')
('SECTION', '6', ' Illness, Medications, and Physical Disabilities ')
('SECTION', '7', ' Sleep Patterns ')
('SECTION', '8', ' Household Roster and Residence History ')
('SECTION', '9', ' Education ')
('SECTION', '10', ' Military ')
('SECTION', '11', ' Labor Market ')
('SECTION', '12', ' Economics ')
('SECTION', '13', ' Religion and Spirituality ')
('SECTION', '14', ' Social Psychology and Mental Health ')
('SECTION', '15', ' Suicide, Sexual Experiences, & Sexually Transmitted Diseases ')
('SECTION', '17', ' Relationship in Detail ')
('SECTION', '21', ' Criminal Offending and Victimization ')
('SECTION', '22', ' Involvement with Criminal Justice System ')
('SECTION', '23', ' Tobacco, Alcohol, Drugs (self-administered by resp

In [ ]:
exclude_prefixes = ['H4RD','H4IR','H4EO']